<h1>Этапы работы<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Работа-с-таблицами" data-toc-modified-id="Работа-с-таблицами-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Работа с таблицами</a></span></li><li><span><a href="#Работа-с-данными" data-toc-modified-id="Работа-с-данными-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Работа с данными</a></span></li><li><span><a href="#Итоговая-таблица" data-toc-modified-id="Итоговая-таблица-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Итоговая таблица</a></span></li></ul></div>

# Работа с таблицами
В данном разделе проведена работа с таблицами: airlines, airports, flights. 
Были объединены три таблицы, удалены столбцы, которые не будут нужны для выполнения заданий, а также переименованы некоторые столбцы для более удобной работы.

In [72]:
import pandas as pd
import numpy as np
import joblib

In [73]:
airlines = pd.read_csv('airlines.csv')
airlines.head(3)

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.


In [74]:
airports = pd.read_csv('airports.csv')
airports.head(3)

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919


In [75]:
airports.columns

Index(['IATA_CODE', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY', 'LATITUDE',
       'LONGITUDE'],
      dtype='object')

In [76]:
flights = pd.read_csv('flights.csv')

C:\Users\acer\AppData\Local\Temp\ipykernel_4208\3005660769.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  flights = pd.read_csv('flights.csv')


In [77]:
flights.head(3)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
flights = flights.drop(columns=['TAIL_NUMBER', 'SCHEDULED_DEPARTURE',
                                'DEPARTURE_TIME', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME',
                                'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'CANCELLED', 'DIVERTED',
                                'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY',
                                'WEATHER_DELAY', 'DAY_OF_WEEK'
                                ])

In [79]:
flights.columns

Index(['YEAR', 'MONTH', 'DAY', 'AIRLINE', 'FLIGHT_NUMBER', 'ORIGIN_AIRPORT',
       'DESTINATION_AIRPORT', 'DEPARTURE_DELAY', 'TAXI_OUT', 'TAXI_IN',
       'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY'],
      dtype='object')

In [80]:
flights.head(3)

,YEAR,MONTH,DAY,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,TAXI_OUT,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY
0,2015,1,1,AS,98,ANC,SEA,-11.0,21.0,4.0,430,408.0,-22.0
1,2015,1,1,AA,2336,LAX,PBI,-8.0,12.0,4.0,750,741.0,-9.0
2,2015,1,1,US,840,SFO,CLT,-2.0,16.0,11.0,806,811.0,5.0


In [81]:
# объединяем три df поочереди: объединяем flights и airlines 
# по колонкам AIRLINE в таблице flights и IATA_CODE в таблице airlines

flights_and_airlines = pd.merge(flights, airlines, left_on='AIRLINE', right_on='IATA_CODE', how='inner')

In [82]:
flights_and_airlines.head(3)

,YEAR,MONTH,DAY,AIRLINE_x,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,TAXI_OUT,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,IATA_CODE,AIRLINE_y
0,2015,1,1,AS,98,ANC,SEA,-11.0,21.0,4.0,430,408.0,-22.0,AS,Alaska Airlines Inc.
1,2015,1,1,AS,135,SEA,ANC,-1.0,11.0,5.0,320,259.0,-21.0,AS,Alaska Airlines Inc.
2,2015,1,1,AS,108,ANC,SEA,-4.0,17.0,4.0,509,455.0,-14.0,AS,Alaska Airlines Inc.


In [83]:
# объединяем flights_and_airlines и airports.
# ORIGIN_AIRPORT и DESTINATION_AIRPORT в таблице flights соответствуют IATA_CODE в таблице airports.

flights_airlines_airports = pd.merge(flights_and_airlines, airports, left_on='ORIGIN_AIRPORT', right_on='IATA_CODE', how='inner')

In [84]:
flights_airlines_airports.head(3)

,YEAR,MONTH,DAY,AIRLINE_x,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,TAXI_OUT,TAXI_IN,...,ARRIVAL_DELAY,IATA_CODE_x,AIRLINE_y,IATA_CODE_y,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,2015,1,1,AS,98,ANC,SEA,-11.0,21.0,4.0,...,-22.0,AS,Alaska Airlines Inc.,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619
1,2015,1,1,AS,108,ANC,SEA,-4.0,17.0,4.0,...,-14.0,AS,Alaska Airlines Inc.,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619
2,2015,1,1,AS,122,ANC,PDX,-4.0,11.0,3.0,...,-18.0,AS,Alaska Airlines Inc.,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619


In [85]:
flights_airlines_airports.columns

Index(['YEAR', 'MONTH', 'DAY', 'AIRLINE_x', 'FLIGHT_NUMBER', 'ORIGIN_AIRPORT',
       'DESTINATION_AIRPORT', 'DEPARTURE_DELAY', 'TAXI_OUT', 'TAXI_IN',
       'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'IATA_CODE_x',
       'AIRLINE_y', 'IATA_CODE_y', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY',
       'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [86]:
flights_airlines_airports['dDATE'] = pd.to_datetime(flights_airlines_airports[['YEAR', 'MONTH', 'DAY']])

In [87]:
df = pd.merge(flights_airlines_airports, airports, left_on='DESTINATION_AIRPORT', right_on='IATA_CODE', how='inner')

In [88]:
df.head(3)

,YEAR,MONTH,DAY,AIRLINE_x,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,TAXI_OUT,TAXI_IN,...,LATITUDE_x,LONGITUDE_x,dDATE,IATA_CODE,AIRPORT_y,CITY_y,STATE_y,COUNTRY_y,LATITUDE_y,LONGITUDE_y
0,2015,1,1,AS,98,ANC,SEA,-11.0,21.0,4.0,...,61.17432,-149.99619,2015-01-01,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931
1,2015,1,1,AS,108,ANC,SEA,-4.0,17.0,4.0,...,61.17432,-149.99619,2015-01-01,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931
2,2015,1,1,AS,136,ANC,SEA,NaN,NaN,NaN,...,61.17432,-149.99619,2015-01-01,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931


In [89]:
df = df.drop(columns=['COUNTRY_y', 'LATITUDE_y', 
                      'LONGITUDE_y', 'COUNTRY_x', 'LATITUDE_x', 'LONGITUDE_x',
                      'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
                     ])

In [90]:
df = df.drop(columns=['STATE_y', 'STATE_x', 'AIRLINE_x', 'IATA_CODE_y', 'IATA_CODE_x'])

In [91]:
df.columns

Index(['YEAR', 'MONTH', 'DAY', 'FLIGHT_NUMBER', 'ORIGIN_AIRPORT',
       'DESTINATION_AIRPORT', 'DEPARTURE_DELAY', 'TAXI_OUT', 'TAXI_IN',
       'ARRIVAL_DELAY', 'AIRLINE_y', 'AIRPORT_x', 'CITY_x', 'dDATE',
       'IATA_CODE', 'AIRPORT_y', 'CITY_y'],
      dtype='object')

In [92]:
df.head(3)

,YEAR,MONTH,DAY,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,TAXI_OUT,TAXI_IN,ARRIVAL_DELAY,AIRLINE_y,AIRPORT_x,CITY_x,dDATE,IATA_CODE,AIRPORT_y,CITY_y
0,2015,1,1,98,ANC,SEA,-11.0,21.0,4.0,-22.0,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Anchorage,2015-01-01,SEA,Seattle-Tacoma International Airport,Seattle
1,2015,1,1,108,ANC,SEA,-4.0,17.0,4.0,-14.0,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Anchorage,2015-01-01,SEA,Seattle-Tacoma International Airport,Seattle
2,2015,1,1,136,ANC,SEA,NaN,NaN,NaN,NaN,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Anchorage,2015-01-01,SEA,Seattle-Tacoma International Airport,Seattle


In [93]:
df.rename(columns={'AIRPORT_x': 'O_AIRPORT'}, inplace=True)
df.rename(columns={'AIRPORT_y': 'D_AIRPORT'}, inplace=True)
df.rename(columns={'AIRLINE_y': 'AIRLINE'}, inplace=True)

In [94]:
df.rename(columns={'CITY_x': 'O_CITY'}, inplace=True)
df.rename(columns={'CITY_y': 'D_CITY'}, inplace=True)

In [95]:
df = df.drop(columns=['IATA_CODE'])

In [96]:
df.head(3)

,YEAR,MONTH,DAY,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,TAXI_OUT,TAXI_IN,ARRIVAL_DELAY,AIRLINE,O_AIRPORT,O_CITY,dDATE,D_AIRPORT,D_CITY
0,2015,1,1,98,ANC,SEA,-11.0,21.0,4.0,-22.0,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Anchorage,2015-01-01,Seattle-Tacoma International Airport,Seattle
1,2015,1,1,108,ANC,SEA,-4.0,17.0,4.0,-14.0,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Anchorage,2015-01-01,Seattle-Tacoma International Airport,Seattle
2,2015,1,1,136,ANC,SEA,NaN,NaN,NaN,NaN,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Anchorage,2015-01-01,Seattle-Tacoma International Airport,Seattle


# Работа с данными

В данном разделе проведена предподготовка данных для выполнения заданий и обучения модели.

In [97]:
df.columns  # проверка наличия всех нужных столбцов для работы

Index(['YEAR', 'MONTH', 'DAY', 'FLIGHT_NUMBER', 'ORIGIN_AIRPORT',
       'DESTINATION_AIRPORT', 'DEPARTURE_DELAY', 'TAXI_OUT', 'TAXI_IN',
       'ARRIVAL_DELAY', 'AIRLINE', 'O_AIRPORT', 'O_CITY', 'dDATE', 'D_AIRPORT',
       'D_CITY'],
      dtype='object')

In [98]:
df.info()  # проверка информации о типах данных

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5332914 entries, 0 to 5332913
Data columns (total 16 columns):
 #   Column               Dtype         
---  ------               -----         
 0   YEAR                 int64         
 1   MONTH                int64         
 2   DAY                  int64         
 3   FLIGHT_NUMBER        int64         
 4   ORIGIN_AIRPORT       object        
 5   DESTINATION_AIRPORT  object        
 6   DEPARTURE_DELAY      float64       
 7   TAXI_OUT             float64       
 8   TAXI_IN              float64       
 9   ARRIVAL_DELAY        float64       
 10  AIRLINE              object        
 11  O_AIRPORT            object        
 12  O_CITY               object        
 13  dDATE                datetime64[ns]
 14  D_AIRPORT            object        
 15  D_CITY               object        
dtypes: datetime64[ns](1), float64(4), int64(4), object(7)
memory usage: 691.7+ MB


In [99]:
# Поиск пропусков в столбцах
missing_values = df.isnull().sum()
missing_values[missing_values > 0]

DEPARTURE_DELAY     83814
TAXI_OUT            86612
TAXI_IN             89942
ARRIVAL_DELAY      101784
dtype: int64

In [100]:
# Поиск столбцов с NuN значениями:
nan_columns = df.columns[df.isnull().any()]
nan_columns

Index(['DEPARTURE_DELAY', 'TAXI_OUT', 'TAXI_IN', 'ARRIVAL_DELAY'], dtype='object')

In [102]:
# Фильтр столбцов по типу данных:
numeric_columns = df.select_dtypes(include=['number']).columns
nan_columns = [col for col in numeric_columns if df[col].isnull().any()]

In [103]:
# Замена NaN значения на медианные значения для каждого из этих столбцов
for column in nan_columns:
  median_value = df[column].median()
  df[column].fillna(median_value, inplace=True)

In [104]:
# Снова проверка столбцов с NuN значениями:
nan_columns_again = df.columns[df.isnull().any()]

In [105]:
nan_columns_again

Index([], dtype='object')

In [106]:
# Проверка на отрицательные значения задержек
anomalies = df[(df['DEPARTURE_DELAY'] < 0) | (df['ARRIVAL_DELAY'] < 0)]

In [107]:
anomalies.head(3)

,YEAR,MONTH,DAY,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,TAXI_OUT,TAXI_IN,ARRIVAL_DELAY,AIRLINE,O_AIRPORT,O_CITY,dDATE,D_AIRPORT,D_CITY
0,2015,1,1,98,ANC,SEA,-11.0,21.0,4.0,-22.0,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Anchorage,2015-01-01,Seattle-Tacoma International Airport,Seattle
1,2015,1,1,108,ANC,SEA,-4.0,17.0,4.0,-14.0,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Anchorage,2015-01-01,Seattle-Tacoma International Airport,Seattle
2,2015,1,1,136,ANC,SEA,-1.0,14.0,6.0,-5.0,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Anchorage,2015-01-01,Seattle-Tacoma International Airport,Seattle


In [108]:
# Исключение аномальных значений:
df = df[(df['DEPARTURE_DELAY'] >= 0) | (df['ARRIVAL_DELAY'] >= 0)]

In [109]:
df

,YEAR,MONTH,DAY,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,TAXI_OUT,TAXI_IN,ARRIVAL_DELAY,AIRLINE,O_AIRPORT,O_CITY,dDATE,D_AIRPORT,D_CITY
15,2015,1,2,108,ANC,SEA,0.0,19.0,4.0,-15.0,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Anchorage,2015-01-02,Seattle-Tacoma International Airport,Seattle
23,2015,1,2,120,ANC,SEA,7.0,9.0,11.0,-5.0,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Anchorage,2015-01-02,Seattle-Tacoma International Airport,Seattle
24,2015,1,2,92,ANC,SEA,9.0,10.0,8.0,-11.0,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Anchorage,2015-01-02,Seattle-Tacoma International Airport,Seattle
29,2015,1,3,108,ANC,SEA,53.0,11.0,4.0,20.0,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Anchorage,2015-01-03,Seattle-Tacoma International Airport,Seattle
30,2015,1,3,136,ANC,SEA,49.0,8.0,9.0,24.0,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Anchorage,2015-01-03,Seattle-Tacoma International Airport,Seattle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5332904,2015,12,26,2734,MEI,PIB,33.0,12.0,4.0,33.0,Atlantic Southeast Airlines,Meridian Regional Airport,Meridian,2015-12-26,Hattiesburg-Laurel Regional Airport,Hattiesburg-Laurel
5332905,2015,12,27,2734,MEI,PIB,93.0,7.0,4.0,89.0,Atlantic Southeast Airlines,Meridian Regional Airport,Meridian,2015-12-27,Hattiesburg-Laurel Regional Airport,Hattiesburg-Laurel
5332906,2015,12,28,2730,MEI,PIB,8.0,11.0,3.0,6.0,Atlantic Southeast Airlines,Meridian Regional Airport,Meridian,2015-12-28,Hattiesburg-Laurel Regional Airport,Hattiesburg-Laurel
5332910,2015,12,30,2730,MEI,PIB,101.0,4.0,5.0,90.0,Atlantic Southeast Airlines,Meridian Regional Airport,Meridian,2015-12-30,Hattiesburg-Laurel Regional Airport,Hattiesburg-Laurel


In [110]:
# Проверка и удаление дубликатов:
df.drop_duplicates(inplace=True)

In [111]:
df

,YEAR,MONTH,DAY,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,TAXI_OUT,TAXI_IN,ARRIVAL_DELAY,AIRLINE,O_AIRPORT,O_CITY,dDATE,D_AIRPORT,D_CITY
15,2015,1,2,108,ANC,SEA,0.0,19.0,4.0,-15.0,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Anchorage,2015-01-02,Seattle-Tacoma International Airport,Seattle
23,2015,1,2,120,ANC,SEA,7.0,9.0,11.0,-5.0,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Anchorage,2015-01-02,Seattle-Tacoma International Airport,Seattle
24,2015,1,2,92,ANC,SEA,9.0,10.0,8.0,-11.0,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Anchorage,2015-01-02,Seattle-Tacoma International Airport,Seattle
29,2015,1,3,108,ANC,SEA,53.0,11.0,4.0,20.0,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Anchorage,2015-01-03,Seattle-Tacoma International Airport,Seattle
30,2015,1,3,136,ANC,SEA,49.0,8.0,9.0,24.0,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Anchorage,2015-01-03,Seattle-Tacoma International Airport,Seattle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5332904,2015,12,26,2734,MEI,PIB,33.0,12.0,4.0,33.0,Atlantic Southeast Airlines,Meridian Regional Airport,Meridian,2015-12-26,Hattiesburg-Laurel Regional Airport,Hattiesburg-Laurel
5332905,2015,12,27,2734,MEI,PIB,93.0,7.0,4.0,89.0,Atlantic Southeast Airlines,Meridian Regional Airport,Meridian,2015-12-27,Hattiesburg-Laurel Regional Airport,Hattiesburg-Laurel
5332906,2015,12,28,2730,MEI,PIB,8.0,11.0,3.0,6.0,Atlantic Southeast Airlines,Meridian Regional Airport,Meridian,2015-12-28,Hattiesburg-Laurel Regional Airport,Hattiesburg-Laurel
5332910,2015,12,30,2730,MEI,PIB,101.0,4.0,5.0,90.0,Atlantic Southeast Airlines,Meridian Regional Airport,Meridian,2015-12-30,Hattiesburg-Laurel Regional Airport,Hattiesburg-Laurel


# Итоговая таблица

В данном разделе проведена создана таблица с окончательными данными, которые будут использованы в работе

In [112]:
joblib.dump(df, 'resulting_data.pkl')

['resulting_data.pkl']